<a href="https://colab.research.google.com/github/Efefefef/IANNwTF-UniOsnabrueck/blob/main/homework07/homework07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK 07

In [760]:
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.layers import Dense, Conv2D, AveragePooling2D, TimeDistributed, LSTM, GlobalAvgPool2D, AbstractRNNCell, MaxPooling2D
from keras.initializers import Orthogonal
from tqdm import tqdm
import datetime

## Prepare Dataset

In [761]:
# Load MNIST dataset
def load_data():
    (train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)
    return (train_ds, test_ds), ds_info

In [762]:
# Creating new target
def new_target_fnc(ds, window_size):
  l = list()
  for i, elem in enumerate(ds):
    if (i % window_size) == 0:
      l.append(int(elem[1]))
    else:
      if (i % 2) == 0:
        l.append(int(l[i-1] + elem[1]))
      else:
        l.append(int(l[i-1] - elem[1]))
  return l

# Preprocessing data
def preprocess(data, batch_size, window_size):
  new_targets = new_target_fnc(data, window_size)
  new_targets = tf.data.Dataset.from_tensor_slices(new_targets)

  data = tf.data.Dataset.zip((data, new_targets))
  data = data.map(lambda img, new_target: (img[0], new_target))
  data = data.map(lambda img, target: (img, tf.one_hot(target, depth=37)))

  data = data.map(lambda img, target: (img, tf.cast(target, tf.float32)))

  data = data.map(lambda img, target: (img,target))

  data = data.map(lambda img, target: (tf.cast(img, tf.float32), target))
  data = data.map(lambda img, target: ((img/128.)-1., target))

  data = data.batch(window_size, drop_remainder=True)
  data = data.batch(batch_size, drop_remainder=True)
  data = data.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
  return data

In [763]:
# CNN Model
class CNN(tf.keras.Model):
  def __init__(self, input_shape):
    super().__init__()
    # input conv1 = 28x28x1
    self.conv1 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'), input_shape=input_shape)
    # output conv1 = 28x28x24
    self.conv2 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    # output conv2 = 28x28x24
    self.pooling1 = TimeDistributed(MaxPooling2D())
    # output pooling1 = 14x14x24
    self.conv3 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    # output conv3 = 14x14x24
    self.conv4 = TimeDistributed(Conv2D(24, 3, activation='relu', padding='valid'))
    # output conv3 = 14x14x24
    self.globalpooling = TimeDistributed(GlobalAvgPool2D())
    # output globalpooling = 7x7x24
    self.out = TimeDistributed(Dense(10, activation="softmax"))


  @tf.function
  def __call__(self, x, training=False):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.pooling1(x)
    x = self.conv3(x)
    x = self.conv4(x)

    x = self.globalpooling(x)
    x = self.out(x)
    return x

In [764]:
class LSTMCell(AbstractRNNCell):
    def __init__(self, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)

        self.hidden_state_units = 50
        self.cell_state_units = 50
        self.input_size = 1

        self.forget_gate = Dense(self.cell_state_units, activation="sigmoid", kernel_initializer=tf.keras.initializers.Orthogonal(gain=1.0, seed=None))
        self.input_gate = Dense(self.cell_state_units, activation="sigmoid", kernel_initializer=tf.keras.initializers.Orthogonal(gain=1.0, seed=None))
        self.cell_state_candidates = Dense(self.cell_state_units, activation="tanh", kernel_initializer=tf.keras.initializers.Orthogonal(gain=1.0, seed=None))
        self.output_gate = Dense(self.hidden_state_units, activation="sigmoid", kernel_initializer=tf.keras.initializers.Orthogonal(gain=1.0, seed=None))

    @property
    def state_size(self):
        return [tf.TensorShape([self.hidden_state_units]), 
                tf.TensorShape([self.cell_state_units])]

    @property
    def output_size(self):
        return [tf.TensorShape([self.hidden_state_units])]

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        return [tf.zeros(shape=[32,self.hidden_state_units]), 
                tf.zeros(shape=[32,self.cell_state_units])]

    def call(self, inputs, states):

        hidden_state = states[0]
        cell_state = states[1]

        hidden_and_input_concat = tf.concat([hidden_state, tf.convert_to_tensor(inputs)], axis=-1)
        hidden_and_input_concat = tf.convert_to_tensor(hidden_and_input_concat)

        forget_filter = self.forget_gate(hidden_and_input_concat)
        input_filter = self.input_gate(hidden_and_input_concat)
        candidate_vector = self.cell_state_candidates(hidden_and_input_concat)

        cell_state_with_forgetting = cell_state * forget_filter
        cell_state = cell_state_with_forgetting + input_filter * candidate_vector

        output_filter = self.output_gate(hidden_and_input_concat)

        tanh_of_cell_state = tf.math.tanh(cell_state)
        hidden_state = output_filter * tanh_of_cell_state
        
        return hidden_state, [hidden_state, cell_state]

In [765]:
class OverallModel(tf.keras.Model):
  def __init__(self, cnn, lstm, optimizer, loss_function):
    super().__init__()

    self.cnn = cnn
    self.lstm = lstm
    self.output_layer = Dense(37,activation='softmax')

    self.metrics_list = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      tf.keras.metrics.Mean(name="loss")]

    self.optimizer = optimizer
    self.loss_function = loss_function

  @property
  def metrics(self):
    return self.metrics_list
    
  def reset_metrics(self):
     for metric in self.metrics:
        metric.reset_state()

  def call(self, sequence, training = False):
    cnn_output = self.cnn(sequence)
    cnn_number = tf.argmax(cnn_output, axis=-1)
    lstm_output = self.lstm(cnn_output)
    output = self.output_layer(lstm_output)
    return output

  @tf.function
  def training_step(self, image, label):

    with tf.GradientTape() as tape: 
      prediction = self(image, training = True)

      loss = self.loss_function(label, prediction)

    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))
    self.metrics[0].update_state(label, prediction)
    self.metrics[1].update_state(loss)  

  @tf.function
  def test_step(self, data):
    image, label = data
    prediction = self(image, training = False)
    loss = self.loss_function(label, prediction)
    self.metrics[0].update_state(label, prediction)
    self.metrics[1].update_state(loss)



In [766]:
#Training LOOp
def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path):
    for epoch in range (epochs):
        model.reset_metrics()

        for data in tqdm(train_ds, position=0, leave=True):

            model.training_step(image=data[0], label=data[1])

        with train_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)
        
        print("Epoch: ", epoch+1)
        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Train)")
        model.reset_metrics()

        for data in test_ds:
            model.test_step(data)

        with test_summary_writer.as_default():
            tf.summary.scalar(model.metrics[0].name, model.metrics[0].result(), step=epoch)
            tf.summary.scalar(model.metrics[1].name, model.metrics[1].result(), step=epoch)

        print("Loss: ", model.metrics[1].result().numpy(), "Accuracy: ", model.metrics[0].result().numpy(), "(Test)")
    
    model.save_weights(save_path)

In [767]:
# train the model

batch_size = 32
window_size = 4
(train_ds,test_ds), ds_info = load_data()
train_ds = preprocess(train_ds, batch_size, window_size)
test_ds = preprocess(test_ds, batch_size, window_size)


In [768]:

optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.CategoricalCrossentropy()
cnn = CNN(input_shape=(window_size, 28, 28, 1))
lstm_cell = LSTMCell()
lstm_layer = tf.keras.layers.RNN(lstm_cell, return_sequences=True, unroll=True)
model = OverallModel(cnn=cnn, lstm=lstm_layer, optimizer=optimizer, loss_function=loss_function)
epochs = 50

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_path = f"models/{current_time}"
train_log_path = f"logs/{current_time}/train"
test_log_path = f"logs/{current_time}/test"
train_summary_writer = tf.summary.create_file_writer(train_log_path)
test_summary_writer = tf.summary.create_file_writer(test_log_path)
training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path)

100%|██████████| 468/468 [00:08<00:00, 54.39it/s]


Epoch:  1
Loss:  2.0286062 Accuracy:  0.13628472 (Train)
Loss:  1.8822297 Accuracy:  0.07652243 (Test)


100%|██████████| 468/468 [00:07<00:00, 63.16it/s]


Epoch:  2
Loss:  1.8460972 Accuracy:  0.14177684 (Train)
Loss:  1.839498 Accuracy:  0.071213946 (Test)


100%|██████████| 468/468 [00:06<00:00, 68.81it/s]


...

Epoch:  48
Loss:  1.797823 Accuracy:  0.16376202 (Train)
Loss:  1.8019049 Accuracy:  0.083834134 (Test)


100%|██████████| 468/468 [00:06<00:00, 67.01it/s]


Epoch:  49
Loss:  1.7975069 Accuracy:  0.15998931 (Train)
Loss:  1.8016461 Accuracy:  0.081430286 (Test)


100%|██████████| 468/468 [00:07<00:00, 66.74it/s]


Epoch:  50
Loss:  1.7973409 Accuracy:  0.16230969 (Train)
Loss:  1.8025831 Accuracy:  0.083834134 (Test)


In [769]:
# Initiating RNN model
rnn = RNNModel()

# Compiling the rnn model
rnn.compile(optimizer, loss_function)

In [770]:
# Training RNN Model 
EXPERIMENT_NAME = "RNN_model"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logging_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/{EXPERIMENT_NAME}/{current_time}")

In [771]:
# Training RNN Model using fit
history = rnn.fit(x= cnn_output_train,
                  y = y_train,
                  validation_data=test_ds,
                  initial_epoch=2,
                  epochs=6,
                  callbacks=[logging_callback])

ValueError: Data cardinality is ambiguous:
  x sizes: 14976
  y sizes: 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3

## Plotting the result

In [ ]:
# Plotting RNN model from History
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(labels=["training","validation"])
plt.xlabel("Epoch")
plt.ylabel("Categorical Crossentropy Loss")
plt.show()